## **Task 1. Classify emails**

It was a simple problem which I solved by using openAI gpt-4 model to understand the semantics of the text and using high level of prompt engineering.

In [71]:
# Install the relevant Python packages
%pip install openai langchain pandas openpyxl langchain-openai

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/re.py", line 293, in _compile
    return _cache[type(pattern), pattern, flags]
KeyError: (<class 're.Pattern'>, re.compile("\n        (?P<operator>(~=|==|!=|<=|>=|<|>|===))\n        \n        (?P<version>\n            (?:\n                # The identity operators allow for an escape hatch that will\n                # do a, re.IGNORECASE|re.VERBOSE), 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-pack

In [123]:
from langchain_openai import OpenAI
import openai
# Define the openAI API KEY to access the model
openai.api_key = #OPENAI API KEY

In [124]:
import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
# display(products_df.head(3))
display(emails_df.head(3))

,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [131]:
def classify_email(email_id,subject,message):
  """
  classify_email() classifies an email as either 'Product Inquiry' or 'Order Request' using the OpenAI GPT-4 model.

  Parameters:
  -email_id (str): The unique identifier for the email.
  -subject (str): The subject line of the email.
  -message (str): The body of the email.

  Returns:
  -tuple: A tuple containing the email_id and the classification of the email.
  """

  # Define the prompt for the model
  message = [
      {"role":"system","content":"You are an assistant tasked with classifying emails into two categories: 'Product Inquiry' or 'Order Request', IMPORTANT: Only output these two categories. Analyze the content of the following message and subject thoroughly to determine the appropriate classification. Consider details such as whether the sender is asking for information about a product, looking for recommendations, or providing specific product details versus placing an actual order. Your classification should reflect the intent of the sender based on the context provided in the email. If an alpha numeric Product ID or Name of product is not found the customer is most probably just 'Product Inquiry'  "},
      {"role":"user","content": f"Subject: {subject}\nMessage: {message}"}
  ]

  # Call the OpenAI API to generate a response
  response = openai.chat.completions.create(
      model='gpt-4',
      messages=message
  )
  # Extract the classification from the response
  category = response.choices[0].message.content.strip()
  # Return the email_id and the classification
  return email_id, category.strip()


# return email_id, category
results = [classify_email(row['email_id'],row['subject'],row['message']) for index, row in emails_df.iterrows()]
# Create a new DataFrame with the results
results_df = pd.DataFrame(results, columns=['email_id', 'category'])

In [132]:
# Prints the head of results_df
print(results_df)

   email_id         category
0      E001    Order Request
1      E002    Order Request
2      E003  Product Inquiry
3      E004    Order Request
4      E005  Product Inquiry
5      E006  Product Inquiry
6      E007    Order Request
7      E008    Order Request
8      E009  Product Inquiry
9      E010    Order Request
10     E011  Product Inquiry
11     E012  Product Inquiry
12     E013  'Order Request'
13     E014    Order Request
14     E015  Product Inquiry
15     E016  Product Inquiry
16     E017  'Order Request'
17     E018    Order Request
18     E019    Order Request
19     E020  Product Inquiry
20     E021  Product Inquiry


In [130]:
# Save the results to an Excel file
results_df.to_excel('email-classification.xlsx', index=False)

## **Task 2. Process order requests**

My approach to Task Two was sequential and well thought of.


---



1.  I started by separating order requests from product inquiries.
2.  I simplified and extracted key information from messages, focusing on essential data such as product ID, name, and quantity. This made the process efficient and optimized for a limited number of orders.
3. After receiving responses in a specific format, I converted them into columns for further processing, including product ID, name, or, if both were missing, the original message.
4. I used advanced regex matching and other techniques to ensure that orders were correctly associated with product logs. I also updated the product log accordingly.
5.  When an order request lacked specific details like the name or product ID, I attempted to match those orders to other fields and descriptions. If enough details were found, the orders were processed. Otherwise, during the email-writing stage for each order request, I included requests for more information from customers to ensure completeness.
6.  Although this additional step could have been incorporated earlier in the process, I included it to test the robustness of the system.

*Part 1.*

In [10]:
# Install the OpenAI Python package.
%pip install openai

In [96]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key=#OPENAI API KEY
)

In [109]:
import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
#display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


In [99]:
import pandas as pd
#read the file into a datafram
df=pd.read_excel('email-classification.xlsx')
#filter the dataframe to only include rows where the category is 'Order Request'
orders_df=df[df['category']=='Order Request']

In [100]:
#Get coresponding message to each email ID which is an Order Request
full_orders_df=pd.merge(orders_df, emails_df[['email_id','message']], on='email_id', how='left')

In [101]:
print(full_orders_df)

   email_id       category                                            message
0      E001  Order Request  Hi there, I want to order all the remaining LT...
1      E002  Order Request  Good morning, I'm looking to buy the VBT2345 V...
2      E004  Order Request  Hi, I'd like to order three to four SFT1098 In...
3      E007  Order Request  Hi, this is Liz. Please send me 5 CLF2109 Cabl...
4      E008  Order Request  Hello, I'd want to order one of your Versatile...
5      E010  Order Request  Hello, I would like to order 1 pair of RSG8901...
6      E013  Order Request  Hi, my name is Marco and I need to buy a pair ...
7      E014  Order Request       Please send me 1 Sleek Wallet. Thanks, Johny
8      E017  Order Request  Hi there I want to place an order for that pop...
9      E018  Order Request  Hello I'd like to buy 2 pairs of the retro sun...
10     E019  Order Request  Hey there, I would like to buy Chelsea Boots [...


In [102]:
# Function to simplify and extract key information from an email
def simplefy_email(email_id, message):
    """
    Simplify and extract key information from an email.

    Parameters:
    -email_id (str): The unique identifier for the email.
    -message (str): The body of the email.

    Returns:
    -tuple: A tuple containing the email_id and the simplified message.
    """
    # Define the prompt for the model
    messages = [
        {"role": "system", "content": "You are an assistant that extracts information from an email message. Extract the 7-digit alphanumeric Product_ID, the Name of the product, and the Description from the email. If a piece of information is not found, return 'null'. Return the result in the format: product_id={Product_ID}, Name={Name}, Description={email content}. For multiple product IDs, separate them using ';'."},
        {"role": "user", "content": f"Message: {message}"}
    ]

    # Call the OpenAI API to generate a response
    try:
        response = client.chat.completions.create(
            model='gpt-4',
            messages=messages
        )

        # Extract the response content
        response_dict = response.model_dump()  # Convert to dictionary
        simplified = response_dict["choices"][0]["message"]["content"].strip()

        # Return the email_id and the simplified message
        return email_id, simplified
    except Exception as e:
        print(f"Error processing email {email_id}: {e}")
        return email_id, None

# Iterate through full_orders_df and simplify each email message
results = [simplefy_email(row['email_id'], row['message']) for index, row in full_orders_df.iterrows()]

# Create a new DataFrame with the results
results_df = pd.DataFrame(results, columns=['email_id', 'simplified'])

# Display the results DataFrame
print(results_df)

   email_id                                         simplified
0      E001  product_id=LTH0976, Name=Leather Bifold Wallet...
1      E002  product_id=VBT2345, Name=Vibrant Tote bag, Des...
2      E004  product_id=SFT1098, Name=Infinity Scarves, Des...
3      E007  product_id=CLF2109;FZZ1098, Name=Cable Knit Be...
4      E008  product_id=null, Name=Versatile Scarves, Descr...
5      E010  product_id=null, Name=RSG8901 Retro Sunglasses...
6      E013  product_id=null, Name=null, Description=Hi, my...
7      E014  product_id=null, Name=Sleek Wallet, Descriptio...
8      E017  product_id=null, Name=null, Description=Hi the...
9      E018  product_id=RSG8901, Name=retro sun glasses, De...
10     E019  product_id=CBT8901;FZZ1098, Name=Chelsea Boots...


In [103]:
results_df.to_excel('email-simplifed.xlsx', index=False)

In [104]:
import re

# Regular expression to capture key-value pairs
pattern = r'product_id=([^,]*), Name=([^,]*), Description=(.*)'

# Split the 'simplified' column into multiple columns
expanded_rows = []

for index, row in results_df.iterrows():
    email_id = row['email_id']
    simplified = row['simplified']

    try:
        # Use regular expression to extract product_id, name, and description
        match = re.match(pattern, simplified)
        if match:
            product_ids = match.group(1).split(';')
            names = match.group(2).split(';')
            description = match.group(3).strip()

            # Create rows for each product
            for i in range(len(product_ids)):
                product_id = product_ids[i] if i < len(product_ids) else 'null'
                name = names[i] if i < len(names) else 'null'
                expanded_rows.append({
                    'email_id': email_id,
                    'product_id': product_id,
                    'name': name,
                    'description': description
                })
        else:
            # If no match, assign null
            expanded_rows.append({
                'email_id': email_id,
                'product_id': 'null',
                'name': 'null',
                'description': 'null'
            })
    except Exception as e:
        print(f"Error processing row {email_id}: {e}")
        expanded_rows.append({
            'email_id': email_id,
            'product_id': 'null',
            'name': 'null',
            'description': 'null'
        })

# Create the new expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Display the expanded DataFrame
print(expanded_df)



   email_id product_id                      name  \
0      E001    LTH0976     Leather Bifold Wallet   
1      E002    VBT2345          Vibrant Tote bag   
2      E004    SFT1098          Infinity Scarves   
3      E007    CLF2109        Cable Knit Beanies   
4      E007    FZZ1098            Fuzzy Slippers   
5      E008       null         Versatile Scarves   
6      E010       null  RSG8901 Retro Sunglasses   
7      E013       null                      null   
8      E014       null              Sleek Wallet   
9      E017       null                      null   
10     E018    RSG8901         retro sun glasses   
11     E019    CBT8901             Chelsea Boots   
12     E019    FZZ1098            Fuzzy Slippers   

                                          description  
0   Hi there, I want to order all the remaining LT...  
1   Good morning, I'm looking to buy the VBT2345 V...  
2   Hi, I'd like to order three to four SFT1098 In...  
3   Hi, this is Liz. Please send me 5 CLF2109 C

In [112]:
%pip install fuzzywuzzy

In [113]:
import pandas as pd
import re
from fuzzywuzzy import process

# Define preprocessing function
def preprocess_name(name):
    """ Normalize the product name by removing special characters and converting to lower case. """
    return re.sub(r'\W+', '', name.lower().strip())

# Define function to create regex pattern
def create_regex_pattern(name):
    """ Create a regex pattern to match the product name allowing for variations. """
    normalized_name = re.sub(r'\W+', '.*', name.strip())
    return f"(?i){normalized_name}"

# Define function to match names using regex
def match_name(name, product_name):
    """ Match a name with a product name using regex. """
    pattern = create_regex_pattern(name)
    return re.search(pattern, preprocess_name(product_name))

# Define function to find best match using fuzzy matching
def find_best_match(name, products_df):
    """ Find the best match for a product name using fuzzy matching. """
    choices = products_df['name'].tolist()
    best_match, score = process.extractOne(name, choices)
    if score > 80:  # Adjust score threshold as needed
        return products_df[products_df['name'] == best_match]['product_id'].values[0]
    return None

# Initialize the order status list and null email IDs list
order_status = []
null_email_ids = []

# Iterate through each row in expanded_df
for i, row in expanded_df.iterrows():
    email_id = row['email_id']
    product_id = row['product_id']
    name = row['name']

    if product_id == 'null' and name == 'null':
        # If both product_id and name are null, store the email ID in null_email_ids
        null_email_ids.append(email_id)
        continue

    if product_id == 'null' and not name == 'null':
        # Attempt to match product name using regex or fuzzy matching
        matched_product_id = None

        # First try regex match
        for j, product_row in products_df.iterrows():
            product_name = product_row['name']
            if match_name(name, product_name):
                matched_product_id = product_row['product_id']
                break

        # If no match found by regex, try fuzzy matching
        if matched_product_id is None:
            matched_product_id = find_best_match(name, products_df)

        # If no match is found by both methods, store the email ID in null_email_ids
        if matched_product_id is None:
            null_email_ids.append(email_id)
            continue

        product_id = matched_product_id

    # Checking if product_id is still 'null' after attempting to match by name
    if product_id == 'null':
        null_email_ids.append(email_id)
        continue

    # Find product information in products_df
    product_info = products_df[products_df['product_id'] == product_id]

    if not product_info.empty:
        # Extract product details
        available_stock = product_info['stock'].values[0]
        product_name = product_info['name'].values[0]

        requested_quantity = 1

        if available_stock >= requested_quantity:
            # Fulfill the order
            status = "created"
            # Update the stock in products_df
            products_df.loc[products_df['product_id'] == product_id, 'stock'] = available_stock - requested_quantity
        else:
            # Order cannot be fully fulfilled
            status = "out of stock"

        # Append the order status
        order_status.append({
            "email_id": email_id,
            "product_id": product_id,
            "quantity": requested_quantity,
            "status": status
        })

    else:
        # If no match is found in products_df
        order_status.append({
            "email_id": email_id,
            "product_id": product_id,
            "quantity": 0,
            "status": "out of stock"
        })

# Create a DataFrame for the order status
order_status_df = pd.DataFrame(order_status, columns=['email_id', 'product_id', 'quantity', 'status'])

# Create a DataFrame for null email IDs
null_email_df = pd.DataFrame(null_email_ids, columns=['email_id'])

# Output the results
print("Order Status DataFrame:")
print(order_status_df)

print("Null Email IDs DataFrame:")
print(null_email_df)




Order Status DataFrame:
   email_id product_id  quantity        status
0      E001    LTH0976         1       created
1      E002    VBT2345         1       created
2      E004    SFT1098         1       created
3      E007    CLF2109         1       created
4      E007    FZZ1098         1       created
5      E008    VSC6789         1       created
6      E010    RSG8901         1       created
7      E014    SWL2345         1       created
8      E018    RSG8901         1  out of stock
9      E019    CBT8901         1       created
10     E019    FZZ1098         1       created
Null Email IDs DataFrame:
  email_id
0     E013
1     E017


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [115]:
#popluate df into order_status.xlsx file
order_status_df.to_excel('order-status.xlsx', index=False)

In [116]:
#for testing
products_df.head()

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,0,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,4,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,5,"Spring, Fall",23.00
3,CSH1098,Cozy Shawl,Accessories,Wrap yourself in comfort with our cozy shawl. ...,3,"Fall, Winter",22.00
4,CHN0987,Chunky Knit Beanie,Accessories,Keep your head toasty with our chunky knit bea...,2,"Fall, Winter",22.00


In [120]:
import pandas as pd

# Function to get product info based on message and product attributes
def getProductInfo(client, email_id, message, product_id, name, description, category, seasons):
    """
    Get product information based on a message and product attributes.


    parameters:
    -client (OpenAI client): The OpenAI client instance.
    -email_id (str): The unique identifier for the email.
    -message (str): The message to be processed.
    -product_id (str): The product ID.
    -name (str): The name of the product.
    -description (str): The description of the product.
    -category (str): The category of the product.
    -seasons (str): The seasons associated with the product.

    Returns:
    -tuple: A tuple containing the email_id and the product information.
    """
    prompt = (
    """You are a helpful assistant tasked with matching product details based on a message.
    Consider the following attributes: Description, Category, and Seasons.
    Match the message with any of these attributes to identify the best product.
    Provide the product ID and name of the best matched product in the format:
    product_id={product_id}, name={name}.

    Here is the information:
    Message: {message}
    Description: {description}
    Product ID: {product_id}
    Name: {name}
    Category: {category}
    Seasons: {seasons}"""
    )


    try:
        # Send the request to LLM
        response = client.chat.completions.create(
            model='gpt-4',
            messages=[{"role": "user", "content": prompt}]
        )
        # Extract the response content
        response_dict = response.to_dict()
        product_info = response_dict["choices"][0]["message"]["content"].strip()

        # Return the extracted information
        return email_id, product_info
    except Exception as e:
        print(f"Error processing email {email_id}: {e}")
        return None


# Collect results
results = []

# Iterate through the emails in null_email_df
for i, row in null_email_df.iterrows():
    email_id = row['email_id']
    print(f"Processing email {email_id}...")

    # Get the corresponding message for this email
    message_row = expanded_df[expanded_df['email_id'] == email_id]
    if not message_row.empty:
        message = message_row['description'].values[0]
    else:
        print(f"No message found for email {email_id}.")
        continue

    # Iterate through products_df
    for j, product_row in products_df.iterrows():
        product_id = product_row['product_id']
        name = product_row['name']
        description = product_row['description']
        category = product_row['category']
        seasons = product_row['seasons']

        print(f"  Checking product {product_id} ({name})...")

        # Semantic matching
        result = getProductInfo(client, email_id, message, product_id, name, description, category, seasons)

        if result:
            print(f"    Match found for email {email_id} and product {product_id}")
            results.append(result)
            break  # Stop checking further products for this email
        else:
            print(f"    No match found for email {email_id} and product {product_id}")

# Create a new DataFrame with the results
ProductID_results_df = pd.DataFrame(results, columns=['email_id', 'product_info'])

# Display or save the resulting DataFrame
print(ProductID_results_df)



Processing email E013...
  Checking product RSG8901 (Retro Sunglasses)...
    Match found for email E013 and product RSG8901
Processing email E017...
  Checking product RSG8901 (Retro Sunglasses)...
    Match found for email E017 and product RSG8901
Processing complete. Here's the resulting DataFrame:
  email_id                                       product_info
0     E013  The provided product details do not match with...
1     E017  Without more specific details in the message a...


*Part 2.*

In [137]:
# Function to generate response emails based on order status or unclear product information
def generate_response(email_id, product_info, product_id, quantity, status, data_source):
    """
    Generate response emails based on order status or unclear product information.


    Parameters:
    -email_id (str): The unique identifier for the email.
    -product_info (str): The product information.
    -product_id (str): The product ID.
    -quantity (int): The quantity of the product.
    -status (str): The order status.
    -data_source (str): The source of the data ('order_status_df' or 'ProductID_results_df').

    Returns:
    -tuple: A tuple containing the email_id and the generated email content.
    """

    if data_source == 'order_status_df':
        # Define the prompt for order statuses
        message = [
            {"role": "system", "content": "You are an assistant who will write emails. If the order status is 'created' or 'out of stock', generate and save response emails based on the order processing results. If the order was successfully processed, inform the customer about the order details including the product name and quantity. If the order could not be fulfilled due to insufficient stock, explain the situation and specify which items are out of stock. Offer options like waiting for restock or choosing alternative products. Ensure the email looks production-ready."},
            {"role": "user", "content": f"Email ID: {email_id}, Product Info: {product_info}, Product ID: {product_id}, Quantity: {quantity}, Status: {status}"}
        ]
    elif data_source == 'ProductID_results_df':
        # Define the prompt for unclear product information
        message = [
            {"role": "system", "content": "You are an assistant who will write emails. If the product info is unclear, inform the customer that their order was not processed due to unclear product details. Include the product info and any details that were provided. Make sure the email looks professional and provides guidance or options for resolving the issue."},
            {"role": "user", "content": f"Email ID: {email_id}, Product Info: {product_info}, Product ID: {product_id}, Quantity: {quantity}, Status: {status}"}
        ]
    else:
        return email_id, "Invalid data source"

    try:
        # Call the OpenAI API to generate a response
        response = client.chat.completions.create(
            model='gpt-4',
            messages=message
        )
        # Extract the email content from the response
        response_dict = response.to_dict()
        # Convert response to dictionary
        email_content = response_dict["choices"][0]["message"]["content"].strip()
        return email_id, email_content
    except Exception as e:
        print(f"Error processing email {email_id}: {e}")
        return email_id, "Error generating response"

# Collect results from both DataFrames
results = []

# Process ProductID_results_df
for index, row in ProductID_results_df.iterrows():
    email_id = row['email_id']
    product_info = row['product_info']
    # Derive or default values for product_id, quantity, and status
    results.append(generate_response(email_id, product_info, None, None, 'unclear', 'ProductID_results_df'))

# Process order_status_df
for index, row in order_status_df.iterrows():
    email_id = row['email_id']
    product_id = row['product_id']
    quantity = row['quantity']
    status = row['status']
    # Derive or default value for product_info
    results.append(generate_response(email_id, None, product_id, quantity, status, 'order_status_df'))

# Creating a DataFrame with the results
email_results_df = pd.DataFrame(results, columns=['email_id', 'email_content'])


Processing complete. Here's the resulting DataFrame:
   email_id                                      email_content
0      E013  Subject: Clarity Required for Your Recent Orde...
1      E017  Subject: Issue With Order Processing: Further ...
2      E001  Subject: Successful Order Confirmation - Produ...
3      E002  Subject: Successful Order Confirmation | Produ...
4      E004  Subject: Order Confirmation for Your Purchase\...
5      E007  Subject: Confirmation on your recent order: CL...
6      E007  Subject: Your Order (FZZ1098) Has Been Success...
7      E008  Subject: Update on Your Recent Order - Confirm...
8      E010  Subject: Your Order - Processing Confirmation ...
9      E014  Subject: Your Order Has Been Successfully Proc...
10     E018  Subject: Your Recent Order Status and Options\...
11     E019  Subject: Acknowledgment of Your Recent Order -...
12     E019  Subject: Your Order has been Successfully Crea...


In [138]:
#Populate the order-response file with df
email_results_df.to_excel('order-response.xlsx', index=False)

## **Task 3.**

My aproch to this problem was to use RAG, Retrieval-Augmented Generation,I Integrate a retrieval component with a generative model to dynamically fetch and generate responses based on relevant parts of the product catalog.


---




*   Import product data using CSVLoader.
*   Manage large texts with RecursiveCharacterTextSplitter.
*   I used FAISS for efficient similarity search.
*   Dynamically fetch relevant information without exceeding token limits.
*   It can handles large catalogs and token limits effectively with FAISS and text splitting.












In [175]:
%pip install langchain_community faiss-cpu langchain langchain-openai

In [143]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# Function to download CSV file from Google Sheets
def download_csv_from_google_sheets(document_id, sheet_name, local_file_path):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    df = pd.read_csv(export_link)
    df.to_csv(local_file_path, index=False)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
sheet_name = 'products'
local_file_path = './products.csv'

download_csv_from_google_sheets(document_id, sheet_name, local_file_path)
loader = CSVLoader(file_path=local_file_path)
data = loader.load()



In [146]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=40)
docs=text_splitter.split_documents(data)

In [176]:
from langchain_openai import ChatOpenAI # Correct path based on documentation
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
#OPENAI_API_KEY=#OPENAI API KEY
#LANGCHAIN_API_KEY=#OPENAI API KEY
#LANGHCHAIN_PROJECT=#OPENAI API KEY

In [152]:
#Vector Embedding and Vector Store
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(docs,OpenAIEmbeddings(api_key=OPENAI_API_KEY))

In [154]:
llm=ChatOpenAI(api_key=OPENAI_API_KEY ,model="gpt-4")

In [159]:
prompt = ChatPromptTemplate.from_template(
""" You are a Helpful customer service employee, Write an email to answer the following questions based only on the provided contxt. Respond to product inquiries using the information from the product catalog
    Think step by step and logically before providing a deatiled answer. I will tip you $1000 if the user finds the asnwer helpful.
    <context>{context}
    <context>
    Question:{input}""")

In [156]:
#chain introduction

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [157]:
## db becomes retiver
retiver = db.as_retriever()


In [158]:
##reiver chain

from langchain.chains import create_retrieval_chain

retrival_chain=create_retrieval_chain(retiver,document_chain)

In [160]:
import pandas as pd
#read the file into a datafram
df=pd.read_excel('email-classification.xlsx')
#filter the dataframe to only include rows where the category is 'Order Request'
inquiry_df=df[df['category']=='Product Inquiry']

In [162]:
#Get coresponding message to each email ID which is an Order Request
full_inquiry_df=pd.merge(inquiry_df, emails_df[['email_id','message']], on='email_id', how='left')

In [177]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"

In [181]:
results = []
for index, row in full_inquiry_df.iterrows():
    email_id = row['email_id']
    question = row['message']


    response = retrival_chain.invoke({
        'input': question
    })


    results.append({
        'email_id': email_id,
        'answer': response['answer']
    })


results_df = pd.DataFrame(results, columns=['email_id', 'answer'])

# Display or save the results
print(results_df)



  email_id                                             answer
0     E003  Hello David,\n\nThe LTH1098 Leather Backpack a...
1     E005  Good day! The Cozy Shawl, product ID CSH1098, ...
2     E006  Hello Sam, the CBT8901 Chelsea Boots would be ...
3     E009  Hola, parece que hubo un error tipográfico en ...
4     E011  The product catalog does not provide specific ...
5     E012  Based on our product catalog, we currently hav...
6     E015  Good morning! Based on your husband's needs, I...
7     E016  Hello Claire, based on your preferences, I hav...
8     E020  Hello Antonio, the Saddle Bag costs $39.00. It...
9     E021  Based on the current product catalog, we do no...


In [183]:
results_df.to_excel('inquiry-response.xlsx', index=False)